<a href="https://colab.research.google.com/github/carlapvalera/LLM-ML/blob/main/source/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering, BertTokenizerFast, AdamW
from datasets import load_dataset
from tqdm import tqdm
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os
from google.colab import drive

# 1. Cargar el dataset SQuAD
print("Cargando el dataset...")
dataset = load_dataset("squad")

# Recortar el dataset para pruebas rápidas
#train_size = 100  # Número de ejemplos de entrenamiento
#eval_size = 20   # Número de ejemplos de evaluación

#dataset["train"] = dataset["train"].select(range(train_size))
#dataset["validation"] = dataset["validation"].select(range(eval_size))

# 2. Cargar el modelo BERT y el tokenizador
print("Cargando el modelo BERT y el tokenizador...")
model_name = "bert-base-uncased"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# 3. Preprocesar los datos
print("Preprocesando los datos...")
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# 4. Preparar los dataloaders
print("Preparando los dataloaders...")
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

# Convertir los datasets a TensorDatasets
def convert_to_tensordataset(dataset):
    print("Convertir los datasets a TensorDatasets")
    return TensorDataset(
        torch.tensor(dataset['input_ids']),
        torch.tensor(dataset['attention_mask']),
        torch.tensor(dataset['start_positions']),
        torch.tensor(dataset['end_positions'])
    )

train_dataset = convert_to_tensordataset(train_dataset)
eval_dataset = convert_to_tensordataset(eval_dataset)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

# 5. Configurar el entrenamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 6. Función de entrenamiento
def process_batch(batch, device):
    if isinstance(batch, list):
        # Si es una lista, asumimos que los elementos están en este orden
        input_ids, attention_mask, start_positions, end_positions = batch
        return {
            'input_ids': input_ids.to(device),
            'attention_mask': attention_mask.to(device),
            'start_positions': start_positions.to(device),
            'end_positions': end_positions.to(device)
        }
    elif isinstance(batch, dict):
        return {k: v.to(device) for k, v in batch.items()}
    else:
        raise ValueError(f"Unexpected batch type: {type(batch)}")

def train(model, dataloader, optimizer, device):
    model.train()
    for batch in tqdm(dataloader, desc="Entrenando"):
        batch = process_batch(batch, device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 7. Función de evaluación
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluando"):
            batch = process_batch(batch, device)
            outputs = model(**batch)
            total_loss += outputs.loss.item()

            # Obtener predicciones
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits
            start_pred = torch.argmax(start_logits, dim=1)
            end_pred = torch.argmax(end_logits, dim=1)

            # Aplanar las predicciones y las etiquetas
            predictions = torch.stack((start_pred, end_pred), dim=1).view(-1).cpu().numpy()
            labels = torch.stack((batch['start_positions'], batch['end_positions']), dim=1).view(-1).cpu().numpy()

            all_predictions.extend(predictions)
            all_labels.extend(labels)

    # Calcular métricas
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='macro')

    return total_loss / len(dataloader), accuracy, f1

# ... (El resto del código permanece igual hasta la sección de entrenamiento y evaluación)


# 8. Entrenamiento y evaluación
num_epochs = 1
print(f"Comenzando entrenamiento por {num_epochs} épocas...")

# Montar Google Drive
drive.mount('/content/drive')

# Crear un directorio en Google Drive para guardar los checkpoints
save_dir = "/content/drive/MyDrive/fine_tuned_bert_squad"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

checkpoint_path = os.path.join(save_dir, "checkpoint.pt")

for epoch in range(num_epochs):
    print(f"Época {epoch + 1}/{num_epochs}")

    # Cargar los pesos de la época anterior si existen
    if epoch > -1 and os.path.exists(checkpoint_path):
        print(f"Cargando pesos de la época anterior")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    train(model, train_dataloader, optimizer, device)
    loss, accuracy, f1 = evaluate(model, eval_dataloader, device)
    print(f"Pérdida de validación: {loss:.4f}")
    print(f"Exactitud de validación: {accuracy:.2%}")
    print(f"F1-score de validación: {f1:.4f}")

    # Guardar los pesos del modelo al final de cada época
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'f1': f1,
    }, checkpoint_path)
    print(f"Checkpoint guardado en {checkpoint_path}")

# Guardar el modelo final en Google Drive
print("Guardando el modelo final...")
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("¡Entrenamiento completado!")

Cargando el dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Cargando el modelo BERT y el tokenizador...


...

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Preparando los dataloaders...
Convertir los datasets a TensorDatasets


Exception ignored in: <function tqdm.__del__ at 0x7efe625a4820>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [3]:
from flask import Flask, request, jsonify
import torch
from transformers import BertForQuestionAnswering, BertTokenizerFast
from google.colab import drive

app = Flask(__name__)

# Montar Google Drive
drive.mount('/content/drive')

# Cargar el modelo y el tokenizador
model_path = "/content/drive/MyDrive/fine_tuned_bert_squad"  # Ruta donde guardaste tu modelo entrenado
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def answer_question(question, context):
    # Tokenizar la pregunta y el contexto
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Obtener predicciones
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Obtener las posiciones de inicio y fin de la respuesta
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    # Convertir las posiciones de los tokens a las posiciones de los caracteres en el contexto
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = tokenizer.convert_tokens_to_string(tokens[answer_start:answer_end+1])

    # Limpiar la respuesta
    answer = answer.replace("[CLS]", "").replace("[SEP]", "").strip()

    return answer

question = "como se llama mi amiguito"
context = "leo es una persomna,Este ejemplo muestra cómo crear una API simple usando Flask para utilizar tu modelo BERT entrenado para responder preguntas. La API acepta solicitudes POST con una pregunta y un contexto, y devuelve la respuesta generada por el modelo. Puedes extender esta API para incluir más funcionalidades según tus necesidades,leo es mi amigo"
answer =answer_question(question, context)
print(answer)

if __name__ == '__main__':
    question = "como se llama mi amiguito"
    context = "leo es una persomna,Este ejemplo muestra cómo crear una API simple usando Flask para utilizar tu modelo BERT entrenado para responder preguntas. La API acepta solicitudes POST con una pregunta y un contexto, y devuelve la respuesta generada por el modelo. Puedes extender esta API para incluir más funcionalidades según tus necesidades,leo es mi amigo"
    print(answer_question(question, context))


Mounted at /content/drive
leo es una persomna, este ejemplo muestra como crear una api simple usando flask para utilizar tu modelo bert entrenado para responder preguntas. la api
leo es una persomna, este ejemplo muestra como crear una api simple usando flask para utilizar tu modelo bert entrenado para responder preguntas. la api


In [4]:
!pip install -U spacy
!python -m spacy download es_core_news_sm
!pip install neo4j
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 98.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Cargar el modelo y el tokenizador
model_name = "t5-small"  # Puedes usar "t5-base" o "t5-large" para mejores resultados, pero requieren más recursos
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def summarize_text(text, max_length=100):
    # Preparar el input
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generar el resumen
    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decodificar y retornar el resumen
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Ejemplo de uso
text = "Este es un texto de ejemplo que se utilizará para generar un resumen usando el modelo T5. El resumen debe capturar las ideas principales del texto de manera concisa y clara."
summary = summarize_text(text)
print("Texto original:")
print(text)
print("\nResumen:")
print(summary)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Texto original:
Este es un texto de ejemplo que se utilizará para generar un resumen usando el modelo T5. El resumen debe capturar las ideas principales del texto de manera concisa y clara.

Resumen:
es un texto de ejemplo que se utilizará para generar un resumen usando el modelo T5.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import threading

def get_similarity(query:str,vector_space:list, k:int) -> list[tuple[str,float]]: #TODO
    """
    Finds the k most relevant texts in a set of texts given a query text.

    Parameters:
        - query (str): The query text.
        - vector_space (list): A list of texts to search.
        - k (int): The number of most relevant texts to return.

    Returns:
        A list of tuples (text, score) where text is one of the k most
        relevant texts and score is its degree of similarity to the query text.
    """
    if vector_space==[]:
        return []
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Ajustar y transformar el texto de consulta y el conjunto de textos
    X = vectorizer.fit_transform([query] + vector_space)

    # Calcular la similitud de coseno entre el texto de consulta y cada texto en el conjunto
    similarities = cosine_similarity(X[0], X[1:])

    # Ordenar los textos por similitud de coseno en orden descendente
    sorted_indices = similarities.argsort()[0][::-1]

    # Devolver los k textos más relevantes
    return [(vector_space[i], similarities[0][i]) for i in sorted_indices[:k]]


class LTM_Node:
    '''
        A node in the LTM network.
        There is two kind of nodes:
            -The actual past conversations:
                These kind of nodes cannot have any children
            -The summary nodes:
                This kind has any number of children in a dictionary (vector->node)
                His vector is just a summary of his children vectors
    '''

    def __init__(self, arg:str|list['LTM_Node'],is_leaf=True):

        if is_leaf:
            '''
                This represent a new memory in LTM
                conversation: the conversation that this node represents
                parents: a list of the nodes that are parents of this node
            '''
            self.vector:str = arg
            self.children: dict[str,'LTM_Node']|None = None
            self.is_leaf:bool = True
            self.parents:list['LTM_Node'] = []
            return

        '''
            This represent a summary of memories
            MemoryList: a list of the memories that this node summarizes (children nodes)
            parents: a list of the nodes that are parents of this node
        '''
        self.children: dict[str,'LTM_Node']|None = {x.vector:x for x in arg}
        self.vector:str = self.calculate_vector()
        self.is_leaf:bool=False
        self.parents=[]

    def insert(self,node:'LTM_Node'):
        '''
            Adds a new children. This node(self) cannot be a memory node
        '''
        if self.children==None:
            raise Exception("LTM Insertion Error")
        else:
            self.children[node.vector]=node
            node.parents.append(self)
        self.update()

    def mix_memories(self, vector:str, new_memory_node:'LTM_Node'):
        '''
            Mix a new memory with a child in a new summary node,
            The new summary will be son of this node instead of the mixed child
            - vector: the vector of the child that will be mixed
            - new_memory_node: the new memory node
        '''
        old_child = self.children[vector]
        new_summary=LTM_Node([old_child,new_memory_node], False)
        new_memory_node.parents.append(new_summary)
        old_child.parents.remove(self)
        old_child.parents.append(new_summary)
        self.children.pop(vector)
        self.insert(new_summary)

    def update(self):
        '''
            Update the vector of this node and the parents of this node
        '''
        if self.children==None:
            return
        self.children={x.vector:x for _, x in self.children.items()}
        self.vector=self.calculate_vector()
        for p in self.parents:
            p.update()

    def get_space(self) -> list[str]|None:
        '''
            Return all the children vectors
        '''
        if self.is_leaf:
            return None
        return list(self.children.keys())

    def calculate_vector(self) -> str:
        """
        Calculate a summary of this node's children using the summarize_text method
        """
        if not self.children:
            return "No hay información para resumir."

        # Combine all texts from children
        combined_text = " ".join([node.text for node in self.children.values()])

        # Use the summarize_text method to generate a summary of the combined texts
        summary = summarize_text(combined_text, max_length=150)

        return summary

class LTM:
    '''
        This class represents the LTM of the model.
        Contains a LTM_Node graph starting with the root node.
        This structure has 2 hyper-parameters:
            - _lambda: the minimum level of relevance that a vector must have for a query
            - k_child: the number of vectors retrieved to a query
        The hyper parameter 'k_child' will also be relevant for the graph construction:
            - For k=1: The graph will be a tree
            - For k>1: The graph will be a DAG
    '''

    def __init__(self, _lambda:float=0.5, k_child:int=1):
        self.root=LTM_Node([], False)
        self._lambda=_lambda
        self.k_child=k_child
        self.where_to_insert=[]
        #NOTE: THE VARIABLE 'where_to_insert' IS USED TO STORAGE A NEW MEMORY,
        # BASED ON THE SEARCH RESULTS HAS THE STRUCTURE [(node,vector)], WHERE:
        #       - node will be the father of the new memory
        #       - vector will be the most relevant vector

    def insert(self,new_memory:str):
        '''
            Insert a new memory in the LTM.

            It will be inserted on the nodes that were relevant for the prompt
            stored in 'where_to_insert' variable updated in 'get_vector' method.
        '''
        new_node = LTM_Node(new_memory)

        for node, vector in self.where_to_insert:
            if node.vector==vector and not vector in node.children.keys():
                #NOTE: Situation 1: The most relevant node was a summary.
                node.insert(new_node)
            else:
                #NOTE: Situation 2: The most relevant node was another memory.
                node.mix_memories(vector,new_node)

        self.where_to_insert=[]

    def get_vector(self, prompt:str) -> list[str]:
        '''
            Return the first k (from 'k_child' variable) most relevant vectors for the prompt

            Also, save the nodes that will be the parents of the new memory and the relevant vectors
            for new memory insertion.
        '''

        self.where_to_insert = [
            (node, vector) for node, vector, _ in
            self.relevant_nodes(self.root, prompt,self._lambda)
        ]

        if len(self.where_to_insert)==0:
            #NOTE: CASE OF NO RELEVANT MEMORIES
            self.where_to_insert=[(self.root,self.root.vector)]
            return ['']

        return [vector for _,vector in self.where_to_insert]

    def relevant_nodes(self, node:LTM_Node, prompt:str,
        _lambda:float, solution=None) -> list[tuple[LTM_Node,str,float]]:
        '''
            Receives a LTM, a prompt and a lambda value
            Return a list of tuple (node, vector, similarity) where:
            - node is the node that gives the future parent vector of the memory
            - vector is the vector that was relevant for the prompt (greater than lambda)
            - similarity is the similarity between the vector and the prompt
            The tuple is also stored in solution
        '''
        if solution==None:
            solution=[]

        best_results = get_similarity(prompt, node.get_space(), self.k_child)

        #The following 3 list are made for a summary node. The i-est element represents each child
        child_node=[]
        child_threads=[]
        child_relevant_nodes=[]

        for vector, similarity in best_results:

            if node.children[vector].is_leaf:
                # On this case the future parent of the memory cannot be the vector itself.
                # So, we will return the parent of the vector and the memories will
                # be mixed into a new summary that will be storage in the parent.
                if similarity>_lambda:
                    solution.append((node,vector, similarity))
                continue

            # If the relevant vector is a summary, we return the best result between this vector and
            # the recursive call on this vector. This process will be running on a thread
            child_node.append((node.children[vector],vector,similarity))
            child_relevant_nodes.append([])
            child_threads.append(threading.Thread(target=self.relevant_nodes,
                args=(node.children[vector], prompt,max(_lambda,similarity), child_relevant_nodes[-1])))
            child_threads[-1].start()

        #Joining the threads
        for i in range(len(child_threads)):
            child_threads[i].join()
            _,_,similarity = child_node[i]
            if len(child_relevant_nodes[i])!=0:
                solution.append(child_relevant_nodes[i][0])
            elif similarity > _lambda:
                solution.append(child_node[i])

        solution.sort(key = lambda x:x[2], reverse=True)
        return solution

In [7]:
from collections import defaultdict
import time

class ShortTermMemory:
    def __init__(self, max_size=100, update_frequency=10, auto_delete_frequency=20):
        self.max_size = max_size
        self.update_frequency = update_frequency
        self.auto_delete_frequency = auto_delete_frequency
        self.memory = defaultdict(lambda: [0, time.time()])

    def add(self, item):
        if item in self.memory:
            self.memory[item][0] += 1
            if self.memory[item][0] % self.auto_delete_frequency == 0:
                del self.memory[item]
        else:
            self.memory[item] = [1, time.time()]
            self._prune_memory()

    def _prune_memory(self):
        if len(self.memory) > self.max_size:
            oldest_items = sorted(self.memory.items(), key=lambda x: x[1][1])
            for item, (count, timestamp) in oldest_items[:-self.max_size]:
                del self.memory[item]

    def update(self):
        current_time = time.time()
        for item, (count, timestamp) in list(self.memory.items()):
            if count % self.update_frequency == 0:
                self.memory[item][1] = current_time
            self.memory[item][0] += 1
            if self.memory[item][0] % self.auto_delete_frequency == 0:
                del self.memory[item]
        self._prune_memory()

# Ejemplo de uso
short_term_memory = ShortTermMemory(max_size=50, update_frequency=5, auto_delete_frequency=10)

short_term_memory.add("apple")
short_term_memory.add("banana")
short_term_memory.add("cherry")
short_term_memory.add("apple")
short_term_memory.add("banana")

print(short_term_memory.memory)
# Output: defaultdict(<function ShortTermMemory.<lambda>.<locals>.<lambda> at 0x7f6a8c0c8d60>, {'apple': [2, 1619540400.0], 'banana': [2, 1619540400.0], 'cherry': [1, 1619540400.0]})

short_term_memory.update()
print(short_term_memory.memory)
# Output: defaultdict(<function ShortTermMemory.<lambda>.<locals>.<lambda> at 0x7f6a8c0c8d60>, {'apple': [3, 1619540405.0], 'banana': [3, 1619540405.0]})

defaultdict(<function ShortTermMemory.__init__.<locals>.<lambda> at 0x7eff07d523b0>, {'apple': [2, 1720580386.4396322], 'banana': [2, 1720580386.4396577], 'cherry': [1, 1720580386.43968]})
defaultdict(<function ShortTermMemory.__init__.<locals>.<lambda> at 0x7eff07d523b0>, {'apple': [3, 1720580386.4396322], 'banana': [3, 1720580386.4396577], 'cherry': [2, 1720580386.43968]})


In [17]:
# Creación de una instancia de LTM
ltm = LTM(_lambda=0.01, k_child=1)

In [16]:
# Función para obtener memoria relevante
def get_relevant_short_term_memory(prompt: str):
    answer = []
    for k, v in short_term_memory.memory:
        answer.append(answer_question(prompt, k))
    return answer

def concatenate_strings(string_list, user_input):
    """
    Concatena una lista de strings.

    Parámetros:
    - string_list (list): La lista de strings a concatenar.
    - user_input (str): La entrada del usuario a añadir al final.

    Devuelve:
    - str: El resultado de concatenar todos los strings en la lista y la entrada del usuario.
    """
    result = ""
    for string in string_list:
        result += string
    result += user_input
    return result
def update_short_term_memory(user_input, response, short_term_memory):
    # Añadir la entrada del usuario a la memoria a corto plazo
    short_term_memory.add(user_input)

    # Añadir la respuesta del LLM a la memoria a corto plazo
    short_term_memory.add(response)

    # Actualizar la memoria a corto plazo
    short_term_memory.update()

In [8]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

class LLM:
    def __init__(self):
        self.model_name = "gpt2"
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.model_name)
        self.model = GPT2LMHeadModel.from_pretrained(self.model_name)

    def generate_response(self, prompt, max_length=100):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = torch.ones(inputs.shape, dtype=torch.long)
        outputs = self.model.generate(inputs,
                                      max_length=max_length,
                                      num_return_sequences=1,
                                      attention_mask=attention_mask,
                                      no_repeat_ngram_size=2,
                                      temperature=0.7)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

def create_prompt(user_input, relevant_short_term_memory, relevant_long_term_memory):
    context = f"Contexto de memoria a corto plazo: {relevant_short_term_memory}\n"
    context += f"Contexto de memoria a largo plazo: {relevant_long_term_memory}\n"
    prompt = f"{context}\nPregunta del usuario: {user_input}\nRespuesta:"
    return prompt

def get_relevant_short_term_memory(user_input):
    # Aquí iría tu lógica para obtener la memoria a corto plazo relevante
    return "hola"


# Inicializar el modelo
llm = LLM()

# Uso
user_input = input("Ingresa tu pregunta: ")

# Obtener las memorias relevantes
relevant_short_term_memory = get_relevant_short_term_memory(user_input)
relevant_long_term_memory = get_relevant_long_term_memory(user_input)

# Crear el prompt y generar la respuesta
prompt = create_prompt(user_input, relevant_short_term_memory, relevant_long_term_memory)
response = llm.generate_response(prompt)

print("Respuesta generada:")
print(response)

Ingresa tu pregunta: hola


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Respuesta generada:
Contexto de memoria a corto plazo: hola
Contexto de memoria a largo plazo: Información relevante a largo plazo basada en la entrada del usuario

Pregunta del usuario: hola
Respuesta:
The following is a list of the most common questions that are asked in the Spanish language.
1. What is the meaning of "the word" in Spanish?
2. How


In [19]:
# 1. Obtener la entrada del usuario
user_input = input("Ingresa tu pregunta: ")

# 2. Inicializar la memoria a corto plazo
short_term_memory = ShortTermMemory(max_size=50, update_frequency=5, auto_delete_frequency=10)

# 3. Obtener el contexto relevante
try:
    relevant_short_term_memory = get_relevant_short_term_memory(user_input)
    print("Contexto obtenido:", context)
except Exception as e:
    print("Error al obtener el contexto:", str(e))
# 3. Extraer lo más relevante de la memoria a largo plazo
relevant_long_term_memory= ltm.get_vector(concatenate_strings(context,user_input))


# 4. Usar todo lo extraído como contexto para generar una respuesta con el LLM
# Inicializar el modelo
llm = LLM()
prompt = create_prompt(user_input, relevant_short_term_memory, relevant_long_term_memory)
response = llm.generate_response(prompt)

# 5. Actualizar la memoria a corto plazo con la respuesta del LLM
update_short_term_memory(user_input, response, short_term_memory)

# 6. Insertar la respuesta en la memoria a largo plazo
new_memory_ltm = concatenate_strings(user_input,response)
print(new_memory_ltm)
ltm.insert(new_memory_ltm)

print(response)

Ingresa tu pregunta: hola
Contexto obtenido: leo es una persomna,Este ejemplo muestra cómo crear una API simple usando Flask para utilizar tu modelo BERT entrenado para responder preguntas. La API acepta solicitudes POST con una pregunta y un contexto, y devuelve la respuesta generada por el modelo. Puedes extender esta API para incluir más funcionalidades según tus necesidades,leo es mi amigo


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


holaContexto de memoria a corto plazo: []
Contexto de memoria a largo plazo: ['']

Pregunta del usuario: hola
Respuesta:
['']"
.
"Prelude"
The following is a list of the most common verbs in the Spanish language. The following verbs are used in Spanish: "to be" (to do), "for" ("to have"), "in"/"out


AttributeError: 'LTM_Node' object has no attribute 'text'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import threading

def get_similarity(query: str, vector_space: list, k: int) -> list[tuple[str, float]]:
    """
    Finds the k most relevant texts in a set of texts given a query text.

    Parameters:
    - query (str): The query text.
    - vector_space (list): A list of texts to search.
    - k (int): The number of most relevant texts to return.

    Returns:
    A list of tuples (text, score) where text is one of the k most
    relevant texts and score is its degree of similarity to the query text.
    """
    if vector_space == []:
        return []

    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Adjust and transform the query text and the set of texts
    X = vectorizer.fit_transform([query] + vector_space)

    # Calculate the cosine similarity between the query text and each text in the set
    similarities = cosine_similarity(X[0], X[1:])

    # Sort the texts by cosine similarity in descending order
    sorted_indices = similarities.argsort()[0][::-1]

    # Return the k most relevant texts
    return [(vector_space[i], similarities[0][i]) for i in sorted_indices[:k]]

def ROUGE(query: str, reference: str, n: int = 1) -> float:
    """
    Calculates the ROUGE score for a query and a reference text.

    Parameters:
    - query (str): The query text.
    - reference (str): The reference text.
    - n (int): The number of n-grams to consider (default is 1).

    Returns:
    The ROUGE score.
    """
    # Tokenize the query and reference texts
    query_tokens = query.split()
    reference_tokens = reference.split()

    # Calculate the number of common n-grams
    common_ngrams = 0
    for i in range(len(query_tokens) - n + 1):
        query_ngram = ' '.join(query_tokens[i:i + n])
        for j in range(len(reference_tokens) - n + 1):
            reference_ngram = ' '.join(reference_tokens[j:j + n])
            if query_ngram == reference_ngram:
                common_ngrams += 1
                break

    # Calculate the ROUGE score
    rouge_score = common_ngrams / (len(query_tokens) - n + 1)

    return rouge_score

def BLEU(query: str, reference: str, n: int = 1) -> float:
    """
    Calculates the BLEU score for a query and a reference text.

    Parameters:
    - query (str): The query text.
    - reference (str): The reference text.
    - n (int): The number of n-grams to consider (default is 1).

    Returns:
    The BLEU score.
    """
    # Tokenize the query and reference texts
    query_tokens = query.split()
    reference_tokens = reference.split()

    # Calculate the number of common n-grams
    common_ngrams = 0
    for i in range(len(query_tokens) - n + 1):
        query_ngram = ' '.join(query_tokens[i:i + n])
        for j in range(len(reference_tokens) - n + 1):
            reference_ngram = ' '.join(reference_tokens[j:j + n])
            if query_ngram == reference_ngram:
                common_ngrams += 1
                break

    # Calculate the BLEU score
    bleu_score = common_ngrams / (len(query_tokens) - n + 1)

    return bleu_score

def METEOR(query: str, reference: str) -> float:
    """
    Calculates the METEOR score for a query and a reference text.

    Parameters:
    - query (str): The query text.
    - reference (str): The reference text.

    Returns:
    The METEOR score.
    """
    # Tokenize the query and reference texts
    query_tokens = query.split()
    reference_tokens = reference.split()

    # Calculate the number of common n-grams
    common_ngrams = 0
    for i in range(len(query_tokens)):
        query_word = query_tokens[i]
        for j in range(len(reference_tokens)):
            reference_word = reference_tokens[j]
            if query_word == reference_word:
                common_ngrams += 1
                break

    # Calculate the METEOR score
    meteor_score = common_ngrams / (len(query_tokens) + len(reference_tokens))

    return meteor_score

# Ejemplo de uso
query = "This is an example sentence."
reference = "This is an example sentence."
n = 1

print("ROUGE-1:", ROUGE(query, reference, n))
print("BLEU-1:", BLEU(query, reference, n))
print("METEOR:", METEOR(query, reference))

ROUGE-1: 1.0
BLEU-1: 1.0
METEOR: 0.5


In [ ]:
from transformers import BertForMaskedLM, BertTokenizer
import torch.nn.functional as F

# Load the model and tokenizer
#model = BertForMaskedLM.from_pretrained('bert-base-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare the input text
input_text = "This is an example sentence."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Calculate the perplexity
output = model(input_ids, labels=input_ids)
loss = output.loss
perplexity = torch.exp(loss)

print(f"Perplexity: {perplexity.item():.2f}")

Perplexity: 46.72


In [ ]:
import torch.nn.functional as F

def cross_entropy(p, q):
    """
    Calculates the cross-entropy between two probability distributions.

    Parameters:
    p (torch.Tensor): The predicted probability distribution.
    q (torch.Tensor): The reference probability distribution.

    Returns:
    float: The cross-entropy value.
    """
    loss = F.cross_entropy(p.view(-1, p.size(-1)), q.view(-1))
    cross_entropy = loss.item()
    return cross_entropy

input_text = "This is an example sentence."
reference_text = "This is a reference sentence."

input_ids = tokenizer.encode(input_text, return_tensors='pt')
reference_ids = tokenizer.encode(reference_text, return_tensors='pt')
output = model(input_ids, labels=input_ids)
p = output.logits  # Distribución de palabras predicha por el modelo
cross_entropy_value = cross_entropy(p, reference_ids)
print(f"Cross-Entropy: {cross_entropy_value:.2f}")

Cross-Entropy: 6.08
